In [1]:
%reload_ext sql
import os

In [2]:
import sqlite3

In [3]:
survey_db_url = 'sqlite:////Users/reethsharma/git/cca/cs513_project/farmers_market.db'
%sql $survey_db_url

In [ ]:
%%sql
SELECT * FROM farmers_market limit 10;

### Next instructions are used to valdiate schema

In [7]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("test") \
      .getOrCreate()

22/06/29 19:57:20 WARN Utils: Your hostname, Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.1.78 instead (on interface en1)
22/06/29 19:57:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/29 19:57:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [22]:
df=spark.read.format("csv").option("header","true").load("dataset/farmers_market.csv",inferSchema = True)

In [20]:
df.createTempView("test")

AnalysisException: Temporary view 'test' already exists

In [11]:
spark.sql("select count(*) from test").show()

+--------+
|count(1)|
+--------+
|    8665|
+--------+



In [13]:
spark.sql("select * from test").head()

Row(FMID='1012063', MarketName=' Caledonia Farmers Market Association - Danville', Website='https://sites.google.com/site/caledoniafarmersmarket/', Facebook='https://www.facebook.com/Danville.VT.Farmers.Market/', Twitter=None, Youtube=None, OtherMedia=None, street=None, city='Danville ', County='Caledonia', State='Vermont', zip='05828', Season1Date='06/08/2016 to 10/12/2016', Season1Time='Wed: 9:00 AM-1:00 PM;', Season2Date=None, Season2Time=None, Season3Date=None, Season3Time=None, Season4Date=None, Season4Time=None, x='-72.140305', y='44.411013', Location=None, Credit='Y', WIC='Y', WICcash='N', SFMNP='Y', SNAP='N', Organic='Y', Bakedgoods='Y', Cheese='Y', Crafts='Y', Flowers='Y', Eggs='Y', Seafood='N', Herbs='Y', Vegetables='Y', Honey='Y', Jams='Y', Maple='Y', Meat='Y', Nursery='N', Nuts='N', Plants='Y', Poultry='Y', Prepared='Y', Soap='Y', Trees='Y', Wine='N', Coffee='Y', Beans='Y', Fruits='Y', Grains='N', Juices='Y', Mushrooms='N', PetFood='Y', Tofu='N', WildHarvested='N', updateTi

In [23]:
df.printSchema()

root
 |-- FMID: integer (nullable = true)
 |-- MarketName: string (nullable = true)
 |-- Website: string (nullable = true)
 |-- Facebook: string (nullable = true)
 |-- Twitter: string (nullable = true)
 |-- Youtube: string (nullable = true)
 |-- OtherMedia: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- Season1Date: string (nullable = true)
 |-- Season1Time: string (nullable = true)
 |-- Season2Date: string (nullable = true)
 |-- Season2Time: string (nullable = true)
 |-- Season3Date: string (nullable = true)
 |-- Season3Time: string (nullable = true)
 |-- Season4Date: string (nullable = true)
 |-- Season4Time: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Credit: integer (nullable = true)
 |-- WIC: integer (nullable = true)
 |-- WIC

In [17]:
spark.sql("CREATE TEMP VIEW test1 AS SELECT * FROM test").show()

++
||
++
++



In [18]:
spark.sql("show create table test1").show()

AnalysisException: test1 is a temp view. 'SHOW CREATE TABLE' expects a table or permanent view.; line 1 pos 18